# e2e-auto-tir

In [1]:
import datetime
import os
import csv
import json
import argparse
import logging
from typing import Dict
import numpy as np  # type: ignore

import tvm
from tvm import relay, relax, runtime, transform
from tvm.ir.module import IRModule
from tvm import meta_schedule as ms
from tvm.meta_schedule.testing.relay_workload import get_network
from tvm.meta_schedule.testing.custom_builder_runner import run_module_via_rpc
from tvm.relax.testing import relay_translator
from tvm.target.target import Target

logging.basicConfig()
logging.getLogger("tvm.meta_schedule").setLevel(logging.DEBUG)

In [8]:
relay_mod, params, (input_name, input_shape, input_dtype) = get_network(
    "wide_resnet_50",
    [1, 3, 224, 224],
    cache_dir="temp",
)

In [9]:
input_info = {input_name: input_shape}
input_data = {}
for input_name, input_shape in input_info.items():
    print(f"\tinput_name: {input_name}")
    print(f"\tinput_shape: {input_shape}")
    print(f"\tinput_dtype: {input_dtype}")

	input_name: input0
	input_shape: [1, 3, 224, 224]
	input_dtype: float32


将 ResNet 模型从 Relay 转换为 Relax。

In [10]:
def apply_opt_before_tuning(
    relay_mod: IRModule, params: Dict[str, runtime.NDArray], target: Target
):
    with transform.PassContext(opt_level=3):
        main_func = relay_mod["main"]
        bind_main_func = relay.build_module.bind_params_by_name(main_func, params)
        relay_mod = IRModule.from_expr(bind_main_func)
        relay_mod = relay.transform.SimplifyInference()(relay_mod)
        relay_mod = relay.transform.FoldConstant()(relay_mod)
        relay_mod = relay.transform.FoldScaleAxis()(relay_mod)
        relay_mod = relay.transform.CanonicalizeOps()(relay_mod)
        relay_mod = relay.transform.AlterOpLayout()(relay_mod)
        relay_mod = relay.transform.FoldConstant()(relay_mod)

        relax_mod = relay_translator.from_relay(relay_mod["main"], target=target)
        relax_mod = relax.transform.AnnotateTIROpPattern()(relax_mod)
        relax_mod = relax.transform.FuseOps()(relax_mod)
        relax_mod = relax.transform.FuseTIR()(relax_mod)
    return relax_mod

In [12]:
relax_mod = apply_opt_before_tuning(relay_mod, params, target="llvm")
assert isinstance(relax_mod, tvm.IRModule)

In [14]:
def f_measurement(
    rt_mod: runtime.Module, device: runtime.ndarray.Device, input_data: dict[str, runtime.NDArray],
    num_measurement_repeats=5, num_measurements=10
):
    vm = relax.VirtualMachine(rt_mod, device=device)
    vm.save_function("main", "measure_func", **input_data, include_return=False)
    evaluator = vm.time_evaluator(
        func_name="measure_func",
        dev=device,
        repeat=num_measurement_repeats,
        number=num_measurements,
        min_repeat_ms=500,
    )
    return evaluator()


def get_runner(target="llvm"):
    target = tvm.target.Target(target)
    if target.attrs.get("mtriple", None) == "aarch64-linux-gnu":
        alloc_repeat = 3
    else:
        alloc_repeat = 1
    runner_config = {
        "evaluator_config": ms.runner.EvaluatorConfig(
            number=3,
            repeat=1,
            min_repeat_ms=100,
            enable_cpu_cache_flush=False,
        ),
        "alloc_repeat": alloc_repeat,
    }
    # runner = ms.runner.RPCRunner(
    #     rpc_config=ARGS.rpc_config, max_workers=ARGS.workers, **runner_config
    # )
    runner = ms.runner.LocalRunner(**runner_config)
    return runner

In [17]:
num_trials = 10
work_dir = "temp/tests"
target = "llvm -num-cores 24"
db = ms.relax_integration.tune_relax(
    mod=relax_mod,
    target=tvm.target.Target(target),
    params=params,
    num_trials_per_iter=64,
    max_trials_per_task=num_trials,
    max_trials_global=num_trials,
    runner=get_runner(target),
    work_dir=work_dir,
)
executable = ms.relax_integration.compile_relax(
    db,
    mod=relax_mod,
    target=target,
    params=params,
)

2023-06-30 10:15:34 [INFO] LocalRunner: max_workers = 1
2023-06-30 10:15:36 [INFO] Logging directory: temp/tests/logs
2023-06-30 10:15:36 [INFO] LocalBuilder: max_workers = 24
2023-06-30 10:15:38 [INFO] [task_scheduler.cc:159] Initializing Task #0: "fused_dense_add13"
2023-06-30 10:15:38 [INFO] [task_scheduler.cc:159] Initializing Task #1: "fused_conv2d20_add10_relu7"
2023-06-30 10:15:38 [INFO] [task_scheduler.cc:159] Initializing Task #2: "fused_conv2d18_add11_add12_relu8"
2023-06-30 10:15:39 [INFO] [task_scheduler.cc:159] Initializing Task #3: "fused_conv2d14_add7_relu5"
2023-06-30 10:15:39 [INFO] [task_scheduler.cc:159] Initializing Task #4: "fused_conv2d17_add10_relu7"
2023-06-30 10:15:39 [INFO] [task_scheduler.cc:159] Initializing Task #5: "fused_conv2d16_add8_relu6"
2023-06-30 10:15:39 [INFO] [task_scheduler.cc:159] Initializing Task #6: "fused_conv2d12_add8_add9_relu6"
2023-06-30 10:15:39 [INFO] [task_scheduler.cc:159] Initializing Task #7: "fused_conv2d13_add8"
2023-06-30 10:15

[10:15:41] /media/pc/data/lxw/ai/tvm/src/meta_schedule/schedule_rule/apply_custom_rule.cc:56: Warning: Unknown schedule rule "meta_schedule.pool_max" for target keys "["cpu"]". Checked PackedFuncs:
  meta_schedule.cpu.meta_schedule.pool_max


2023-06-30 10:15:42 [INFO] [task_scheduler.cc:159] Initializing Task #18: "fused_conv2d6_add4_relu3"
2023-06-30 10:15:42 [INFO] [task_scheduler.cc:159] Initializing Task #19: "fused_conv2d1_add1_relu1"
2023-06-30 10:15:42 [INFO] [task_scheduler.cc:159] Initializing Task #20: "fused_layout_transform1_reshape_squeeze"
2023-06-30 10:15:42 [INFO] [task_scheduler.cc:159] Initializing Task #21: "fused_conv2d5_add2_relu2"
2023-06-30 10:15:42 [INFO] [task_scheduler.cc:159] Initializing Task #22: "fused_contrib_conv2d_winograd_without_weight_transform_add1_relu1"
2023-06-30 10:15:43 [INFO] [task_scheduler.cc:159] Initializing Task #23: "adaptive_avg_pool2d"


[10:15:43] /media/pc/data/lxw/ai/tvm/src/meta_schedule/schedule_rule/apply_custom_rule.cc:56: Warning: Unknown schedule rule "meta_schedule.adaptive_pool_avg" for target keys "["cpu"]". Checked PackedFuncs:
  meta_schedule.cpu.meta_schedule.adaptive_pool_avg


2023-06-30 10:15:43 [INFO] [task_scheduler.cc:159] Initializing Task #24: "fused_conv2d7_add5_add6_relu4"
2023-06-30 10:15:43 [INFO] [task_scheduler.cc:159] Initializing Task #25: "fused_conv2d4_add1_relu1"
2023-06-30 10:15:43 [INFO] [task_scheduler.cc:159] Initializing Task #26: "fused_conv2d8_add5"
2023-06-30 10:15:44 [INFO] [task_scheduler.cc:159] Initializing Task #27: "fused_conv2d2_add2_add3_relu2"
2023-06-30 10:15:44 [INFO] [task_scheduler.cc:159] Initializing Task #28: "fused_conv2d9_add4_relu3"


...

2023-06-30 10:15:54 [DEBUG] [task_scheduler.cc:318] 
 ID |                                                               Name |      FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
  0 |                                                  fused_dense_add13 |   4097000 |      1 |        73.7709 |      55.5368 |               55.5368 |     10 |    Y 
  1 |                                         fused_conv2d20_add10_relu7 | 205621248 |      2 |            N/A |          N/A |                   N/A |      0 |    Y 
  2 |                                   fused_conv2d18_add11_add12_relu8 | 205821952 |      3 |            N/A |          N/A |                   N/A |      0 |    Y 
  3 |                                          fused_conv2d14_add7_relu5 | 205721600 |      5 |            N/A |

,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,fused_dense_add13,4097000,1,73.7709,55.5368,55.5368,10,Y
1,fused_conv2d20_add10_relu7,205621248,2,N/A,N/A,N/A,0,Y
2,fused_conv2d18_add11_add12_relu8,205821952,3,N/A,N/A,N/A,0,Y
3,fused_conv2d14_add7_relu5,205721600,5,N/A,N/A,N/A,0,Y
4,fused_conv2d17_add10_relu7,924944384,1,N/A,N/A,N/A,0,Y
5,fused_conv2d16_add8_relu6,411443200,1,N/A,N/A,N/A,0,Y
6,fused_conv2d12_add8_add9_relu6,206123008,6,N/A,N/A,N/A,0,Y
7,fused_conv2d13_add8,205721600,1,N/A,N/A,N/A,0,Y
8,fused_conv2d19_add11,205621248,1,N/A,N/A,N/A,0,Y
9,fused_conv2d10_add5_relu4,411844608,1,N/A,N/A,N/A,0,Y


2023-06-30 10:15:54 [DEBUG] [task_scheduler.cc:318] 
 ID |                                                               Name |      FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
  0 |                                                  fused_dense_add13 |   4097000 |      1 |        73.7709 |      55.5368 |               55.5368 |     10 |    Y 
  1 |                                         fused_conv2d20_add10_relu7 | 205621248 |      2 |            N/A |          N/A |                   N/A |      0 |    Y 
  2 |                                   fused_conv2d18_add11_add12_relu8 | 205821952 |      3 |            N/A |          N/A |                   N/A |      0 |    Y 
  3 |                                          fused_conv2d14_add7_relu5 | 205721600 |      5 |            N/A |

,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,fused_dense_add13,4097000,1,73.7709,55.5368,55.5368,10,Y
1,fused_conv2d20_add10_relu7,205621248,2,N/A,N/A,N/A,0,Y
2,fused_conv2d18_add11_add12_relu8,205821952,3,N/A,N/A,N/A,0,Y
3,fused_conv2d14_add7_relu5,205721600,5,N/A,N/A,N/A,0,Y
4,fused_conv2d17_add10_relu7,924944384,1,N/A,N/A,N/A,0,Y
5,fused_conv2d16_add8_relu6,411443200,1,N/A,N/A,N/A,0,Y
6,fused_conv2d12_add8_add9_relu6,206123008,6,N/A,N/A,N/A,0,Y
7,fused_conv2d13_add8,205721600,1,N/A,N/A,N/A,0,Y
8,fused_conv2d19_add11,205621248,1,N/A,N/A,N/A,0,Y
9,fused_conv2d10_add5_relu4,411844608,1,N/A,N/A,N/A,0,Y


2023-06-30 10:15:54 [DEBUG] [task_scheduler.cc:318] 
 ID |                                                               Name |      FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
  0 |                                                  fused_dense_add13 |   4097000 |      1 |        73.7709 |      55.5368 |               55.5368 |     10 |    Y 
  1 |                                         fused_conv2d20_add10_relu7 | 205621248 |      2 |            N/A |          N/A |                   N/A |      0 |    Y 
  2 |                                   fused_conv2d18_add11_add12_relu8 | 205821952 |      3 |            N/A |          N/A |                   N/A |      0 |    Y 
  3 |                                          fused_conv2d14_add7_relu5 | 205721600 |      5 |            N/A |

In [18]:
for input_name, input_shape in input_info.items():
    if input_dtype.startswith("float"):
        input_data[input_name] = np.random.uniform(size=input_shape).astype(input_dtype)
    else:
        input_data[input_name] = np.random.randint(
            low=0, high=10000, size=input_shape, dtype=input_dtype
        )
# for documentation purposes
start_time = datetime.datetime.now()

# RPC
# -----
# ms.runner.RPCConfig(
#     tracker_host=parsed.rpc_host,
#     tracker_port=parsed.rpc_port,
#     tracker_key=parsed.rpc_key,
#     session_timeout_sec=parsed.rpc_timeout_sec,
# )
# workers = rpc_config.count_num_servers(allow_missing=False)
# result = run_module_via_rpc(
#     rpc_config=ARGS.rpc_config,
#     lib=executable.mod,
#     dev_type=ARGS.target.kind.name,
#     args=input_data,
#     continuation=f_measurement,
# )
# ----

dev = tvm.device(tvm.target.Target(target).kind.name)
result = f_measurement(executable.mod, dev, input_data)

print(result)

# out_path = os.path.abspath(os.path.expanduser(ARGS.results_file))
# with open(out_path, "w") as out_file:
#     writer = csv.writer(out_file)
#     # write experiment parameters at the top as a record
#     writer.writerow(["start", str(start_time)])
#     writer.writerow(["workload", ARGS.workload])
#     writer.writerow(["input_shape", ARGS.input_shape])
#     writer.writerow(["target", ARGS.target])
#     writer.writerow(["num_measurement_repeats", ARGS.num_measurement_repeats])
#     for res in result.results:
#         writer.writerow([str(res)])

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
 36618.7247   35841.1667   38509.4056   35412.7562   1292.3571                 
